In [270]:
import random
#生成节点集
#总节点数nodeCount
#每个节点的子节点数kidNodeCount上限
#没单金额上下限orderAmountMax,orderAmountMin
#层级数
def buildNodes(nodeCount,kidNodeCount,orderAmountMax,orderAmountMin,levelCount):
    #随机生成节点集，子节点包括sale 和customer
    pos = 0
    listRandom.clear()
    listLevelCode.clear()
    #listNodes.clear()

    i= 0 
    while i< levelCount: 
        levelCode = chr(66+i)
        listLevelCode.append(levelCode)
        i += 1

    i = 0
    while i < nodeCount-1:
        n = random.choice(listLevelCode) #本节点层级
        if n==listLevelCode[0]:
            p = pos
            pos += 1
            if pos>kidNodeCount-1:
                n = random.choice(listLevelCode[1:]) 
                p = random.randint(0,kidNodeCount-1)
        else:
            p = random.randint(0,kidNodeCount-1) #父节点位置，每个节点最多有40个子节点
        
        if n == listLevelCode[len(listLevelCode)-1]: 
            #叶子节点
            cs = 0
        else:
            cs = random.randint(0,1) #0:customer 1:sale

        if cs == 0:
            v = round(random.uniform(orderAmountMin,orderAmountMax),2) 
        else:
            v = round(random.uniform(0,orderAmountMax),2) #本节点自购金额
        if v<orderAmountMin:
            v = 0.00
        
        #children 子节点集，sonSun子节点value合计，descentSum 孙节点value合计
        #叶子节点不含以上三个数据
        if cs == 0:
            node = {"name":n,"pos":p,"value":v,"cs":0}
        else:
            node = {"name":n,"pos":p,"value":v,"children":[],"rule10":0.00,"rule11":0.00,"rule2":v,"subCount":1,"subSaleCount":1,"cs":1}
    
        listRandom.append(node)
        i += 1
    return
  
#按名称排序
def takeName(elem):
    return elem["name"]

#按位置排序
def takePos(elem):
    return elem["pos"]

In [271]:
#查找同一层级节点
def findLevelNodes(listAllNode,level):
    listLevel = [];
    for elem in listAllNode:
        if elem["name"] == level:
            listLevel.append(elem)
            
    #按在父节点中的顺序排序
    listLevel.sort(key=takePos)
    
    #修改节点名称（按统一level中的顺序命名，例如C1，C2，C3。。。）
    updName(listLevel)
    
    return listLevel

#查找同一层级节点(仅含sale)
def findLevelSaleNodes(listAllNode,level):
    listLevel = [];
    for elem in listAllNode:
        if elem["name"] == level and elem["cs"] == 1:
            listLevel.append(elem)
            
    #按在父节点中的顺序排序
    listLevel.sort(key=takePos)

    return listLevel

#修改节点名称
def updName(listLevel):
    pos = 0
    for elem in listLevel:
        name = elem["name"]
        p = elem["pos"]
        if name == "B":
            name =name+str(elem["pos"])
        else:
            name = name + str(pos)
            pos +=1
        elem["name"] = name
        
    return

In [272]:
#在树中查找父节点,并插入children中
def findParent(elem,listUpLevel):
    result = 0
    name = elem["name"]
    pos = elem["pos"]

    #父节点名称
    parent = chr(ord(name[0])-1)+str(pos)
    
    #遍历上一级节点，添加到父节点的children列表中
    for node in listUpLevel:
        if node["cs"] == 0: continue
        if node["name"] == parent:
            #找到父节点，修改父节点的子节点
            #print("found parent node at pos")
            updLevelSaleData(node,elem)
            result = 1
            break
            
    #父级节点中未找到，添加到pos+1节点
    if result == 0:
        m=len(listUpLevel)
        pos +=1
        parent = chr(ord(name[0])-1)+str(pos)
        while (pos < m and result == 0):
            for node in listUpLevel:
                if node["cs"] == 0: continue
                if node["name"] == parent:
                    #找到父节点，修改父节点的子节点
                    #print("found parent node at another pos")
                    updLevelSaleData(node,elem)
                    elem["pos"] = pos
                    result = 1
                    break 
            pos += 1
            parent = chr(ord(name[0])-1)+str(pos)
                
               
    #父级节点中未找到，添加到父级中第一一个节点                
    if result == 0:
        for node in listUpLevel:
            if node["cs"] == 1:
                n = node["name"]
                pos = int(n[1:])
                #print("found parent node at %d" % pos)
                updLevelSaleData(node,elem)
                elem["pos"] = pos
                break
    
    #将父节点名称保存在子节点中
    pname = chr(ord(name[0])-1)+str(elem["pos"])
    elem["pNode"] = pname
    return listUpLevel  

#统计上下级销售数据      
def updLevelSaleData(parent,kid):  
    name = kid["name"]
    v = kid["value"]
    if kid["cs"] == 1:
        #kid是销售
        r0 = kid["rule10"]
        sc = kid["subSaleCount"]
        c = kid["subCount"]
        parent["rule11"] += r0
        parent["subSaleCount"] +=sc
        parent["subCount"] += c
    else:
        parent["subCount"] += 1
 
    parent["rule10"] += v
    parent["rule2"] += v   
    
    #保留两位小数
    parent["rule10"] = round(parent["rule10"] ,2)
    parent["rule11"] = round(parent["rule11"] ,2)
    parent["rule2"] = round(parent["rule2"] ,2)

    listKid = parent["children"]
    listKid.append(kid)

    return


print("the function has been built")

the function has been built


In [273]:
#计算根节点
def countRoot(node,listKids):
#     node = treeData[0]
#     listKids = node["children"]
    v = 0.00
    vs = 0.00
    sc = 0.00
    c = 0.00
    for elem in listKids:
        v += elem["value"]
        if elem["cs"] == 1:
            vs += elem["rule10"]
            sc += elem["subSaleCount"]
            c += elem["subCount"]
        else:
            c += 1
        elem["pNode"] = "A"
            
    node["rule10"] = v
    node["rule11"] = vs
    node["rule2"] += v
    node["subCount"] += c
    node["subSaleCount"] += sc

    #保留两位小数
    node["rule10"] = round(node["rule10"] ,2)
    node["rule11"] = round(node["rule11"] ,2)
    node["rule2"] = round(node["rule2"] ,2)

    return node

In [274]:
#保存原始生成的节点集
listRandom=[]

#用于保存整理name，父节点名后的节点集
#listNodes=[]

#将listRandom整理成用于画树形图图的节点集
treeData=[]

#用A、B、C。。。。命名各层级
listLevelCode = []

In [275]:
#1:建立节点
nodeCount =499;kidNodeCount=40;orderAmountMax=26000;orderAmountMin=339;levelCount=7
buildNodes(nodeCount,kidNodeCount,orderAmountMax,orderAmountMin,levelCount)

#2: 按照层级排序    
listRandom.sort(key = takeName) 

In [276]:
#3:从叶子节点开始，查找父节点并插入sale父节点的children列表
#同时整理listNodes
i = levelCount
listLeave = findLevelNodes(listRandom,listLevelCode[i-1])

while i > 1:
    uplevel = listLevelCode[i-2]
    listUpLevel = findLevelNodes(listRandom,uplevel)
    for elem in listLeave:
        #父级所有节点，并统计销售数据
        findParent(elem,listUpLevel)   
    listLeave = listUpLevel
    i -= 1

In [277]:
#4: 生成根节点
v = round(random.uniform(0,orderAmountMax),2) #本节点自购金额
if v<orderAmountMin:
    v = 0.00
node = {"name":"A","pos":0,"value":v,"rule10":0.00,"rule11":0.00,"rule2":v,"subCount":1,"subSaleCount":1,"cs":1,"pNode":""}

#5:统计根节点数据
node = countRoot(node,listUpLevel)
node["children"] = listUpLevel
treeData.append(node)
#print(treeData)

In [278]:
#计算单个节点佣金
def calcNodeCommission(node):
    #规则一： 第一代20%佣金，第二代10%
    c1 = node["rule10"]*0.2 + node["rule11"]*0.1

    #规则二：团队大于3万 5%，[1万，3万) 4%, [5千，1万） 3%， 小于5千 2%
    c2=0
    if node["rule2"]>=30000:
        c2 = node["rule2"]*0.05
    elif node["rule2"]>=10000 and node["rule2"]<30000:
         c2 = node["rule2"]*0.04
    elif node["rule2"]>=10000 and node["rule2"]<30000:
         c2 = node["rule2"]*0.03
    else:
         c2 = node["rule2"]*0.02

    #规则三：团队大于100名销售，500元算一单，总业绩》=10万 ，一单 20元，小于10万，一单 15
    c = node["value"]+node["rule10"]+node["rule11"]
    c3 = 0
    ord = c//500
    if node["subSaleCount"]>98:
        if c3>=100000:
            c3 = ord * 20
        else:
            c3 = ord *15

    #规则四：团队50名销售，于30万 奖励15000，[20万，30万) 10000, [20千，10万） 5000，  [5千，10万） 2500，
    c4=0
    if node["subSaleCount"]>98:
        if c >=300000:
            c4 = 15000
        elif c >=200000 and c <300000:
            c4 = 10000
        elif c >=100000 and c <200000:
            c4 = 5000
        elif c >=50000 and c <100000:
            c4 = 2500
        else:
            c4=0

    commission = c1+c2+c3+c4 
        
    return commission


In [279]:
import queue
import copy

listAfterComm = [];
orderTotol = 0.00
commTotal = 0.00

#6:计算佣金
listComm = copy.deepcopy(treeData)

que = queue.Queue()
node = listComm[0]

listKids = node.pop("children")
que.put(listKids)

value = round(node["value"],2)
orderTotol += value
comm = calcNodeCommission(node)
comm = round(comm,2)
commTotal += comm
node["commission"] = comm

listAfterComm.append(node)
while not que.empty():
    listKids = que.get()
    for node in listKids:
        value = round(node["value"],2)
        orderTotol += value
        if "children" in node:
            listk = node.pop("children")
            que.put(listk)
        if node["cs"] == 1:
            comm = calcNodeCommission(node)
            comm = round(comm,2)
            commTotal += comm
            node["commission"] = comm
        listAfterComm.append(node)
    listKids.clear()
print(orderTotol)
print(commTotal)

2513807.2
908845.36


In [280]:
import csv

#5:生成的节点数据导出到csv
# 列表中含有列表时用deepcopy才能完全复制list
f = open('saleTeamWithComm.csv','w')
with f:
    w = csv.DictWriter(f, listAfterComm[0].keys())
    for node in listAfterComm:
        if node["name"][0] == "A":
            w.writeheader()
        w.writerow(node)
f.close()


In [281]:
from pyecharts import options as opts
from pyecharts.charts import Tree
from pyecharts.commons.utils import JsCode

rate = (commTotal/orderTotol)*100
titlestr = "模拟销售层级 营业额：%.2f 佣金合计：%.2f 佣金比例：%.2f" % (orderTotol,commTotal,rate)
print(titlestr)
myTree = (
    Tree(init_opts=opts.InitOpts(
            #设置动画
            animation_opts=opts.AnimationOpts(animation_delay=1000, animation_easing="elasticOut"),
            #设置宽度、高度
            width='7000px',
            height='1000px', 
        ))
    .add("", treeData,initial_tree_depth=7,orient="TB",
         pos_top = '1%',
          pos_left = '3%',
          pos_bottom = '1%',
          pos_right = '2%',
         symbol="circle",
        label_opts=opts.LabelOpts(
            is_show = True,
            position="bottom",
            horizontal_align="middle",
            vertical_align="middle",
            formatter='{b}',
            #formatter='{b}\n 自购:{c}',
            #rotate=-90,
        ),)
    #.set_global_opts(title_opts=opts.TitleOpts(title="模拟销售层级"))
    .set_global_opts(title_opts=opts.TitleOpts(title=titlestr),tooltip_opts=opts.TooltipOpts(axis_pointer_type="shadow",
            formatter=JsCode(                  #formatter为标签内容格式器{a}：系列名;{b}：数据名;{c}：数值数组也可以是回调函数    
                    """function(params) {
                        if (params.data.cs == 0){
                            return '客户 自购:' + params.data.value.toFixed(2);
                        }else{
                             return '销售 自购:' + params.data.value.toFixed(2) + ' <br/>第一级合计:' + params.data.rule10.toFixed(2) + '<br/>第二级合计:' + params.data.rule11.toFixed(2);
                        }
                }""")))
)

myTree.render("tree_base.html")
#myTree.render_notebook()

模拟销售层级 营业额：2513807.20 佣金合计：908845.36 佣金比例：36.15


'/Users/vivianwoo/devProjects/jupyter/tree_base.html'

In [203]:
import csv
import queue
import copy

#5:生成的节点数据导出到csv
# 列表中含有列表时用deepcopy才能完全复制list
listCsv = copy.deepcopy(treeData)

que = queue.Queue()
node = listCsv[0]

listKids = node.pop("children")

que.put(listKids)


f = open('saleTeam.csv','w')
with f:
    w = csv.DictWriter(f, node.keys())
    w.writeheader()
    w.writerow(node)
    while not que.empty():
        listKids = que.get()

        for node in listKids:
            #print(node)
            if "children" in node:
                listk = node.pop("children")
                que.put(listk)
            w.writerow(node)
        listKids.clear()
